<a href="https://colab.research.google.com/github/jdmartinev/ArtificialIntelligenceIM/blob/main/Lecture09/notebooks/L09_text_rep_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea de clasificación de texto

Como hemos mencionado, nos enfocaremos en una tarea simple de clasificación de texto basada en el conjunto de datos **AG_NEWS**, que consiste en clasificar titulares de noticias en una de 4 categorías: Mundo, Deportes, Negocios y Ciencia/Tecnología.

## El Conjunto de Datos

Este conjunto de datos está integrado en el módulo [`torchtext`](https://github.com/pytorch/text), por lo que podemos acceder a él fácilmente.

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# In order to use GPU memory cautiously, we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
dataset = tfds.load('ag_news_subset')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.9PYGGG_1.0.0/ag_news_subset-train.tfrecord*...: …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.9PYGGG_1.0.0/ag_news_subset-test.tfrecord*...:  …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


Ahora podemos acceder a las porciones de entrenamiento y prueba del conjunto de datos usando `dataset['train']` y `dataset['test']` respectivamente:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Imprimamos los primeros 10 titulares de noticias de nuestro conjunto de datos:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vectorización de texto

Ahora necesitamos convertir el texto en **números** que puedan ser representados como tensores. Si queremos una representación a nivel de palabras, necesitamos hacer dos cosas:

* Usar un **tokenizador** para dividir el texto en **tokens**.
* Construir un **vocabulario** de esos tokens.

### Limitando el tamaño del vocabulario

En el ejemplo del conjunto de datos AG News, el tamaño del vocabulario es bastante grande, con más de 100,000 palabras. En términos generales, no necesitamos palabras que estén raramente presentes en el texto &mdash; solo unas pocas oraciones las tendrán, y el modelo no aprenderá de ellas. Por lo tanto, tiene sentido limitar el tamaño del vocabulario a un número menor pasando un argumento al constructor del vectorizador:

Ambos pasos pueden ser manejados utilizando la capa **TextVectorization**. Vamos a instanciar el objeto vectorizador y luego llamar al método `adapt` para recorrer todo el texto y construir un vocabulario:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Nota**: estamos usando solo un subconjunto de todo el conjunto de datos para construir el vocabulario. Hacemos esto para acelerar el tiempo de ejecución y evitar que tengas que esperar. Sin embargo, corremos el riesgo de que algunas palabras del conjunto de datos completo no se incluyan en el vocabulario y se ignoren durante el entrenamiento. Por lo tanto, usar el tamaño completo del vocabulario y recorrer todo el conjunto de datos durante la fase de `adapt` debería aumentar la precisión final, aunque no de manera significativa.

Ahora podemos acceder al vocabulario actual:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Usando el vectorizador, podemos codificar fácilmente cualquier texto en un conjunto de números:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1])>

## Representación de texto usando bolsa de palabras

Debido a que las palabras representan significado, a veces podemos deducir el significado de un texto simplemente observando las palabras individuales, independientemente de su orden en la oración. Por ejemplo, al clasificar noticias, palabras como *clima* y *nieve* probablemente indican *pronóstico del tiempo*, mientras que palabras como *acciones* y *dólar* apuntarían hacia *noticias financieras*.

La representación vectorial **bolsa de palabras** (BoW) es la más sencilla de entender entre las representaciones vectoriales tradicionales. Cada palabra está vinculada a un índice vectorial, y un elemento del vector contiene el número de apariciones de cada palabra en un documento dado.

![Imagen que muestra cómo se representa una bolsa de palabras en la memoria.](images/bag-of-words-example.png)

> **Nota**: También puedes pensar en BoW como una suma de todos los vectores codificados en *one-hot* para las palabras individuales en el texto.

A continuación, se muestra un ejemplo de cómo generar una representación de bolsa de palabras utilizando la biblioteca de Python Scikit Learn:


In [8]:
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

También podemos usar el vectorizador de Keras que definimos anteriormente, convirtiendo cada número de palabra en una codificación *one-hot* y sumando todos esos vectores:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Nota**: Puede que te sorprenda que el resultado sea diferente al ejemplo anterior. La razón es que, en el ejemplo de Keras, la longitud del vector corresponde al tamaño del vocabulario, que fue construido a partir de todo el conjunto de datos AG News, mientras que en el ejemplo de Scikit Learn construimos el vocabulario a partir del texto de muestra de forma dinámica.


## Entrenamiento del clasificador BoW

Ahora que hemos aprendido cómo construir la representación de bolsa de palabras de nuestro texto, entrenemos un clasificador que la utilice. Primero, necesitamos convertir nuestro conjunto de datos a una representación de bolsa de palabras. Esto se puede lograr utilizando la función `map` de la siguiente manera:


In [10]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Ahora definamos una red neuronal clasificadora simple que contenga una capa lineal. El tamaño de entrada es `vocab_size`, y el tamaño de salida corresponde al número de clases (4). Dado que estamos resolviendo una tarea de clasificación, la función de activación final es **softmax**:


In [11]:
# Define the model using Input
model = keras.models.Sequential([
    keras.layers.Input(shape=(vocab_size,)),  # Define the input shape explicitly
    keras.layers.Dense(4)
])

# Compile the model with the appropriate loss function for logits
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['acc'])

# Train the model
model.fit(ds_train_bow, validation_data=ds_test_bow)

938/938 ━━━━━━━━━━━━━━━━━━━━ 48s 48ms/step - acc: 0.7821 - loss: 0.8257 - val_acc: 0.8687 - val_loss: 0.4414


Dado que tenemos 4 clases, una precisión superior al 80% es un buen resultado.

## Entrenando un clasificador como una red completa

Debido a que el vectorizador también es una capa de Keras, podemos definir una red que lo incluya y entrenarla de manera completa. De esta forma, no necesitamos vectorizar el conjunto de datos usando `map`, solo podemos pasar el conjunto de datos original como entrada de la red.

> **Nota**: Todavía tendríamos que aplicar `map` a nuestro conjunto de datos para convertir campos de diccionarios (como `título`, `descripción` y `etiqueta`) a tuplas. Sin embargo, al cargar datos desde el disco, podemos construir un conjunto de datos con la estructura requerida desde el principio.


In [12]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

class SumLayer(keras.layers.Layer):
    def call(self, x):
        return tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = SumLayer()(x)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization                   │ (None, None)                │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sum_layer (SumLayer)                 │ (None, 5335)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │          21,344 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,344 (83.38 KB)

 Trainable params: 21,344 (83.38 KB)

 Non-trainable params: 0 (0.00 B)

938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - acc: 0.7752 - loss: 0.8168 - val_acc: 0.8729 - val_loss: 0.4197


## Bigramas, trigramas y n-gramas

Una limitación del enfoque de bolsa de palabras es que algunas palabras forman parte de expresiones de varias palabras. Por ejemplo, la palabra "hot dog" tiene un significado completamente diferente de las palabras "hot" y "dog" en otros contextos. Si representamos las palabras "hot" y "dog" siempre con los mismos vectores, puede confundir a nuestro modelo.

Para abordar esto, las **representaciones n-gram** se utilizan a menudo en métodos de clasificación de documentos, donde la frecuencia de cada palabra, bi-palabra o tri-palabra es una característica útil para entrenar clasificadores. En las representaciones de bigramas, por ejemplo, añadimos todos los pares de palabras al vocabulario, además de las palabras originales.

A continuación, se muestra un ejemplo de cómo generar una representación de bolsa de palabras de bigramas utilizando Scikit Learn:


In [13]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

La principal desventaja del enfoque de n-gramas es que el tamaño del vocabulario comienza a crecer extremadamente rápido. En la práctica, necesitamos combinar la representación n-gram con una técnica de reducción de dimensionalidad, como los *embeddings*, que discutiremos en la próxima unidad.

Para usar una representación n-gram en nuestro conjunto de datos **AG News**, necesitamos pasar el parámetro `ngrams` al constructor de `TextVectorization`. ¡La longitud de un vocabulario de bigramas es **significativamente mayor**! En nuestro caso, tiene más de 1.3 millones de tokens. Por lo tanto, tiene sentido limitar los tokens de bigramas a un número razonable.

Podríamos usar el mismo código que arriba para entrenar el clasificador, sin embargo, sería muy ineficiente en términos de memoria. En la próxima unidad, entrenaremos el clasificador de bigramas utilizando embeddings. Mientras tanto, puedes experimentar con el entrenamiento del clasificador de bigramas en este cuaderno y ver si logras obtener una mayor precisión.


## Cálculo automático de vectores BoW

En el ejemplo anterior calculamos los vectores BoW manualmente sumando las codificaciones *one-hot* de palabras individuales. Sin embargo, la última versión de TensorFlow nos permite calcular vectores BoW automáticamente pasando el parámetro `output_mode='count'` al constructor del vectorizador. Esto hace que definir y entrenar nuestro modelo sea significativamente más fácil:


In [14]:
# Definir el modelo con un vectorizador y una capa densa sin softmax
model = keras.models.Sequential([
    # TextVectorization capa con BoW (output_mode='count')
    keras.layers.TextVectorization(max_tokens=vocab_size, output_mode='count'),
    # Capa densa sin softmax, que producirá los logits directamente
    keras.layers.Dense(4)  # No usamos softmax aquí
])

# Adaptar el vectorizador a una muestra del conjunto de entrenamiento
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))

# Compilar el modelo con la pérdida que acepta logits
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['acc'])

# Entrenar el modelo
model.fit(ds_train.map(tupelize).batch(batch_size), validation_data=ds_test.map(tupelize).batch(batch_size))


Training vectorizer
938/938 ━━━━━━━━━━━━━━━━━━━━ 45s 47ms/step - acc: 0.7875 - loss: 0.8026 - val_acc: 0.8779 - val_loss: 0.4167


## Frecuencia de términos - frecuencia inversa de documentos (TF-IDF)

En la representación de bolsa de palabras (BoW), las ocurrencias de palabras se ponderan utilizando la misma técnica independientemente de la palabra en sí. Sin embargo, es evidente que palabras frecuentes como *a* o *en* son mucho menos importantes para la clasificación que términos especializados. En la mayoría de las tareas de PLN, algunas palabras son más relevantes que otras.

**TF-IDF** significa **frecuencia de términos - frecuencia inversa de documentos**. Es una variación de la bolsa de palabras, donde en lugar de un valor binario 0/1 que indica la aparición de una palabra en un documento, se utiliza un valor en punto flotante relacionado con la frecuencia de aparición de la palabra en el corpus.

Más formalmente, el peso $w_{ij}$ de una palabra $i$ en el documento $j$ se define como:
$$
w_{ij} = tf_{ij}\times\log\left(\frac{N}{df_i}\right)
$$
donde:
* $tf_{ij}$ es el número de apariciones de $i$ en $j$, es decir, el valor de BoW que hemos visto antes.
* $N$ es el número de documentos en la colección.
* $df_i$ es el número de documentos que contienen la palabra $i$ en toda la colección.

El valor de TF-IDF $w_{ij}$ aumenta proporcionalmente al número de veces que una palabra aparece en un documento y se ajusta por el número de documentos en el corpus que contienen la palabra, lo que ayuda a corregir el hecho de que algunas palabras aparecen con más frecuencia que otras. Por ejemplo, si la palabra aparece en *todos* los documentos de la colección, $df_i=N$, y $w_{ij}=0$, por lo que esos términos se ignorarían completamente.

Puedes crear fácilmente la vectorización TF-IDF de texto utilizando Scikit Learn:


In [15]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

En Keras, la capa `TextVectorization` puede calcular automáticamente las frecuencias TF-IDF pasando el parámetro `output_mode='tf-idf'`. Repitamos el código que usamos anteriormente para ver si el uso de TF-IDF aumenta la precisión:


In [19]:
model = keras.models.Sequential([
    keras.layers.TextVectorization(max_tokens=vocab_size, output_mode='tf-idf'),
    keras.layers.Dense(4)  # No es necesario pasar input_shape, se infiere automáticamente
])

print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
# Compilar el modelo con la pérdida que acepta logits
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 ━━━━━━━━━━━━━━━━━━━━ 45s 48ms/step - acc: 0.8118 - loss: 0.5640 - val_acc: 0.8874 - val_loss: 0.3421


## Conclusión

Aunque las representaciones TF-IDF proporcionan pesos de frecuencia a diferentes palabras, no son capaces de representar el significado o el orden. Como dijo el famoso lingüista J. R. Firth en 1935, "El significado completo de una palabra siempre es contextual, y ningún estudio de significado fuera de contexto puede tomarse en serio". Más adelante en el curso aprenderemos cómo capturar información contextual del texto utilizando modelos de lenguaje.
